In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt


julia_roberts_wiki_url = 'https://en.wikipedia.org/wiki/Julia_Roberts_filmography'
page = urlopen(julia_roberts_wiki_url)
soup = BeautifulSoup(page)
table=soup.find('table', class_='wikitable plainrowheaders sortable')
base_url = 'https://en.wikipedia.org/'



names=[]
counter=[]


# get all available links for all movie pages
movies_url=[]
for row in table.findAll("tr"):
    headers=row.findAll('th')
    if(len(headers)==1):
        link = headers[0].find(href=True)
        if(link is not None):
            movies_url.append(base_url + link.get('href'))
            
            
def getSoupFromUrl(url):
    try:
        page = urlopen(url)
    except:
        return 'failed'
    return BeautifulSoup(page)
    
def setArrayWithNameAndCount(actor_url):
    try:
        soup = getSoupFromUrl(actor_url)
        table = soup.find('table', class_='infobox')
        if table is None:
            pass
        else:
            #find only the name of the actor
            first_row = table.findAll("tr")[0]
            header = first_row.findAll('th')
            name = header[0].find(text=True)
            if name is None:
                name='Name not found'
            if name == 'Julia Roberts':
                pass
            else:
                try:    
                    if name not in names:
                        names.append(name)
                        counter.append(1)
                    else:
                        index = names.index(name)
                        counter[index] += 1
                except ValueError:
                    pass
    except:
        pass

def extract_names_from_ul(ul):
    actor_urls=[]
    for li in ul.findAll('li'):
        actor_url = li.find(href=True)
        if actor_url is not None :
            actor_urls.append(base_url + actor_url.get('href'))
    for actor_url in actor_urls:
        setArrayWithNameAndCount(actor_url)
        
        
def extract_names_from_table(table):
    actor_urls=[]
    for row in table.findAll("tr"):
        cells=row.findAll('td')
        if(len(cells)==3 or len(cells)==2):
            actor_url = cells[0].find(href=True)
            if actor_url is not None:
                actor_urls.append(base_url + actor_url.get('href'))
        
    if len(actor_urls)==0:
       print('cells lenght problem')

    for actor_url in actor_urls:
        setArrayWithNameAndCount(actor_url)

def createHistogram():
    """"
    count_set = set()
    for element in counter:
        count_set.add(element)
    set_as_list = list(count_set)
    values = []
    for val in counter:
        if val in values:
            index = values.index(val)
            values[index] += 1
        else:
            values.append(1)
    
    # np.histogram(values, bins=set_as_list)
    plt.hist(values, bins=set_as_list)
    plt.show()
    """
    plt.hist(counter, bins=10, facecolor='blue', alpha=0.5)
    plt.title("Co-actors/actress per number of joint movie")
    plt.show()
    
for url in movies_url:
    try:
        actor_urls=[]
        soup = getSoupFromUrl(url)
        if soup != 'fail':
            cast_span_tag = soup.find('span', class_='mw-headline', text=re.compile('(cast)|(Cast)'))
            if hasattr(cast_span_tag, 'parent'):
                header_tag= cast_span_tag.parent
                actors = header_tag.findNext(['ul','table'])
                if(actors.name=='ul'):
                    extract_names_from_ul(actors)
                else:
                    extract_names_from_table(actors)


    except ValueError or AttributeError as error:
        pass

df=pd.DataFrame(names,columns=['Name'])
df['Number of joint']=counter
df

createHistogram()
